Data Cleaning, remove empty strings/values and replace them with None/Nan

In [2]:
import pandas as pd
import numpy as np

Method to replace empty cells with none

In [3]:
def replace_empty_with_none(df: pd.DataFrame) -> pd.DataFrame:
    """
    Replaces all empty or whitespace-only strings in a DataFrame with NaN (None).
    """
    cleaned_df = df.copy()
    cleaned_df = cleaned_df.replace(r'^\s*$', np.nan, regex=True)
    return cleaned_df


Tests for replacing empty

In [4]:
def test_replace_empty_with_none():
    # arrange
    df = pd.DataFrame({
        "A": ["", " ", "Hello", None]
    })

    # act
    cleaned = replace_empty_with_none(df)

    # assert
    assert isinstance(cleaned, pd.DataFrame), "Output should be a pandas DataFrame"

    assert cleaned.shape == df.shape, "Shape of DataFrame should not change"

    # pd.isna(value) → checks if a value (or values) is missing (i.e., NaN, None, or NaT)
    assert pd.isna(cleaned.loc[0, "A"]), "Row 0, Col A should be NaN"
    assert pd.isna(cleaned.loc[1, "A"]), "Row 1, Col A should be NaN"

    assert cleaned.loc[2, "A"] == "Hello", "Non-empty text should remain the same"
   
    nan_count = cleaned.isna().sum().sum()
    assert nan_count == 3, f"Expected 3 NaN values, got {nan_count}"

    print("✅ All assertions passed — function works correctly!")

# run it
test_replace_empty_with_none()

✅ All assertions passed — function works correctly!


Check if there are any duplicates with the same ID

In [5]:
def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove duplicates from the DataFrame
    """
    cleaned_df = df.drop_duplicates(subset=["ID"])
    return cleaned_df


Tests for method to check for duplicates with the same ID

In [6]:
test_df = pd.DataFrame({
    "ID": [101, 101],
    "Owner’s Name": ["Michiel", "Hennuyères"],
    "City/Region ": ["Paris", "Lyon"],
    "Sale-Price (€)": [250000, 310000],
    "  Date of-Sale ": ["2025-01-15", "2025-03-10"]
})

print("🧾 Original dataframe:")
print(test_df)

# === Remove duplicates ===
cleaned_df = remove_duplicates(test_df)

print("\nCleaned dataframe:")
print(cleaned_df)

🧾 Original dataframe:
    ID Owner’s Name City/Region   Sale-Price (€)   Date of-Sale 
0  101      Michiel        Paris          250000      2025-01-15
1  101   Hennuyères         Lyon          310000      2025-03-10

Cleaned dataframe:
    ID Owner’s Name City/Region   Sale-Price (€)   Date of-Sale 
0  101      Michiel        Paris          250000      2025-01-15


Transform the headers/titles from csv need to be transformed into snakecase. Because then it is easier to select the columns in dataframe... 

df.property_id
vs
df["Property ID]

In [7]:
import pandas as pd
import re
import unicodedata

def normalize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a copy of the DataFrame with normalized, snake_case column names.
    - Removes accents
    - Converts to lowercase
    - Replaces spaces and symbols with underscores
    - Removes non-alphanumeric characters
    - Collapses multiple underscores
    """
    def clean(col):
        # Normalize accents (é → e)
        col = unicodedata.normalize('NFKD', col)
        col = ''.join(c for c in col if not unicodedata.combining(c))
        
        # Lowercase and replace separators
        col = col.lower()
        col = re.sub(r"[ \-()/.,:;+]", "_", col)
        
        # Remove remaining special characters
        col = re.sub(r"[^0-9a-z_]", "", col)
        
        # Collapse multiple underscores and trim edges
        col = re.sub(r"_+", "_", col).strip("_")
        
        return col

    df_copy = df.copy()
    df_copy.columns = [clean(str(col)) for col in df_copy.columns]
    return df_copy




In [8]:
test_df = pd.DataFrame({
    "Property ID (Ref#)": [101, 102],
    "Owner’s Name": ["Michiel", "Hennuyères"],
    "City/Region ": ["Paris", "Lyon"],
    "Sale-Price (€)": [250000, 310000],
    "  Date of-Sale ": ["2025-01-15", "2025-03-10"]
})

print("🧾 Original columns:")
print(test_df.columns.tolist())

# === Normalize ===
clean_df = normalize_column_names(test_df)

print("\nCleaned columns:")
print(clean_df.columns.tolist())

print("\nCleaned dataframe:")
print(clean_df.head())

print("\nSee the columns")

print(clean_df.columns)

print("\nSelect the column:")

print(clean_df["property_id_ref"])
print("\nOr select it this way:")
print(clean_df.property_id_ref)



🧾 Original columns:
['Property ID (Ref#)', 'Owner’s Name', 'City/Region ', 'Sale-Price (€)', '  Date of-Sale ']

Cleaned columns:
['property_id_ref', 'owners_name', 'city_region', 'sale_price', 'date_of_sale']

Cleaned dataframe:
   property_id_ref owners_name city_region  sale_price date_of_sale
0              101     Michiel       Paris      250000   2025-01-15
1              102  Hennuyères        Lyon      310000   2025-03-10

See the columns
Index(['property_id_ref', 'owners_name', 'city_region', 'sale_price',
       'date_of_sale'],
      dtype='object')

Select the column:
0    101
1    102
Name: property_id_ref, dtype: int64

Or select it this way:
0    101
1    102
Name: property_id_ref, dtype: int64


Add column with price per m2

In [12]:
def calculate_price_per_m2_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the price per m2 of the living area
    """
    added_price_per_m2 = df.copy()
    if "Price per m2" in added_price_per_m2.columns:
        print("Column 'Price per m2' already exists.")
    else:
        added_price_per_m2["Price per m2"] = added_price_per_m2["Price"] / added_price_per_m2["Living area"]
        print("Added column 'Price per m2'.")
    display(added_price_per_m2.head())
    return added_price_per_m2

test_df_price_per_m2 = pd.read_csv("../../clean_data.csv")
calculate_price_per_m2_column(test_df_price_per_m2)
calculate_price_per_m2_column(test_df_price_per_m2)

Added column 'Price per m2'.


,Property ID,Locality name,Postal code,Price,Type of property,Subtype of property,Type of sale,Number of rooms,Living area,Equipped kitchen,Furnished,Open fire,Terrace,Garden,Number of facades,Swimming pool,State of building,Garden Surface,Terrace Surface,Price per m2
0,VWD15538,Seneffe,7180,410000.0,House,Villa,Sale,4.0,175.0,1.0,0,0,1,1,4.0,0,Normal,NaN,NaN,2342.857143
1,RBU63284,Hennuyères,7090,750000.0,House,House,Sale,4.0,550.0,NaN,0,0,1,1,4.0,0,Normal,1085.0,30.0,1363.636364
2,VBD47252,La Louvière,7100,175000.0,Appartment,Flat,Sale,2.0,76.0,1.0,0,0,1,0,2.0,0,NaN,NaN,NaN,2302.631579
3,RBU62400,Solre-sur-Sambre,6560,80000.0,House,House,Sale,3.0,235.0,NaN,0,0,1,0,3.0,0,To be renovated,NaN,30.0,340.425532
4,RBU62399,Mons,7000,195000.0,House,House,Sale,2.0,103.0,NaN,0,0,1,1,NaN,0,NaN,80.0,10.0,1893.203883


Added column 'Price per m2'.


,Property ID,Locality name,Postal code,Price,Type of property,Subtype of property,Type of sale,Number of rooms,Living area,Equipped kitchen,Furnished,Open fire,Terrace,Garden,Number of facades,Swimming pool,State of building,Garden Surface,Terrace Surface,Price per m2
0,VWD15538,Seneffe,7180,410000.0,House,Villa,Sale,4.0,175.0,1.0,0,0,1,1,4.0,0,Normal,NaN,NaN,2342.857143
1,RBU63284,Hennuyères,7090,750000.0,House,House,Sale,4.0,550.0,NaN,0,0,1,1,4.0,0,Normal,1085.0,30.0,1363.636364
2,VBD47252,La Louvière,7100,175000.0,Appartment,Flat,Sale,2.0,76.0,1.0,0,0,1,0,2.0,0,NaN,NaN,NaN,2302.631579
3,RBU62400,Solre-sur-Sambre,6560,80000.0,House,House,Sale,3.0,235.0,NaN,0,0,1,0,3.0,0,To be renovated,NaN,30.0,340.425532
4,RBU62399,Mons,7000,195000.0,House,House,Sale,2.0,103.0,NaN,0,0,1,1,NaN,0,NaN,80.0,10.0,1893.203883


,Property ID,Locality name,Postal code,Price,Type of property,Subtype of property,Type of sale,Number of rooms,Living area,Equipped kitchen,Furnished,Open fire,Terrace,Garden,Number of facades,Swimming pool,State of building,Garden Surface,Terrace Surface,Price per m2
0,VWD15538,Seneffe,7180,410000.0,House,Villa,Sale,4.0,175.0,1.0,0,0,1,1,4.0,0,Normal,NaN,NaN,2342.857143
1,RBU63284,Hennuyères,7090,750000.0,House,House,Sale,4.0,550.0,NaN,0,0,1,1,4.0,0,Normal,1085.0,30.0,1363.636364
2,VBD47252,La Louvière,7100,175000.0,Appartment,Flat,Sale,2.0,76.0,1.0,0,0,1,0,2.0,0,NaN,NaN,NaN,2302.631579
3,RBU62400,Solre-sur-Sambre,6560,80000.0,House,House,Sale,3.0,235.0,NaN,0,0,1,0,3.0,0,To be renovated,NaN,30.0,340.425532
4,RBU62399,Mons,7000,195000.0,House,House,Sale,2.0,103.0,NaN,0,0,1,1,NaN,0,NaN,80.0,10.0,1893.203883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7281,VBD46386,Schaarbeek,1030,300000.0,Appartment,Flat,Sale,2.0,81.0,NaN,0,0,1,0,NaN,0,Normal,NaN,3.0,3703.703704
7282,VBD46380,Ukkel,1180,1050000.0,Appartment,Flat,Sale,4.0,240.0,1.0,1,0,1,0,NaN,1,Excellent,NaN,20.0,4375.000000
7283,VBD46378,Sint-Gillis,1060,745000.0,House,House,Sale,6.0,235.0,1.0,1,0,1,1,2.0,0,Fully renovated,50.0,25.0,3170.212766
7284,VBD46355,Vorst,1190,540000.0,House,House,Sale,4.0,240.0,1.0,0,0,1,1,2.0,0,To be renovated,160.0,16.0,2250.000000
